In [1]:
! pip install transformers
! pip install tree_sitter==0.2.0
! git clone -q https://github.com/microsoft/CodeXGLUE.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.4/110.4 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tree_sitter: filename=tree_sitter-0.2.0-cp311-cp311-linux_x86_64.whl size=389537 sha256=397a4619fc9db5808549449d3922f584046e517e4d56338f530b964f024c5639
  Stored in directory: /root/.cache/pip/wheels/d9/6e/e2/b0126ad4f531cf09749b69518118f0ebf7bf3134ed91c71abb
Successfully built tree_sitter


In [10]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [11]:
# Load data

import pandas as pd

# Make sure to update path to reflect the layout of your google drive
test = pd.read_csv('/content/drive/MyDrive/AIFSD Lab 3/Homework 2/evaluation_data/test_data.csv')

test

,Unnamed: 0,formatted_method,if_statement,predicted_statement
0,0,"def read(self, count=True, timeout=None, ignor...",if ignore_timeouts and is_timeout(e):,if ignore_timeouts and is_noerr(e):
1,1,"def _cache_mem(curr_out, prev_mem, mem_len, re...",if prev_mem is None:,if prev_mem is None:
2,2,def filtered(gen): for example in gen: example...,if example_len > max_length:,if example_len > max_length:
3,3,"def search(self, query): if not query: logger....","if item.get('type', '') == 'audio':",if 'guide_id' in item:
4,4,"def _check_script(self, script, directive): fo...",if var.must_contain('/'):,if not var.can_match(directive):
...,...,...,...,...
4982,4982,"def _super_function(args): passed_class, passe...","if isinstance(pyclass, pyobjects.AbstractClass):","if isinstance(pyclass, pyobjects.PyObject):"
4983,4983,"def get_data(row): data = [] for field_name, f...",if result:,if result:
4984,4984,"def say(jarvis, s): """"""Reads what is typed.""""""...",if not voice_state:,if voice_state:
4985,4985,"def __import__(name, globals=None, locals=None...",if '*' in fromlist:,if '*' in fromlist:


In [55]:
import os
import subprocess
import re

def write_code_to_files(method, target, prediction, target_file, prediction_file):
  with open(target_file, 'w') as f:
    f.write(method.replace("<IF-STMT>", target))

  with open(prediction_file, 'w') as f:
    f.write(method.replace("<IF_STMT>", prediction))




def get_codeBLEU_score(method, target, prediction, target_file, prediction_file):

  write_code_to_files(method, target, prediction, target_file, prediction_file)

  try:
    result = subprocess.run(
        ['python', 'calc_code_bleu.py', '--refs', target_file, '--hyp', prediction_file, '--lang', 'python', '--params', '0.25,0.25,0.25,0.25'],
        cwd = "CodeXGLUE/Code-Code/code-to-code-trans/evaluator/CodeBLEU/",
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        check=True,
        text=True,
        )
    # Since the regular BLEU score is the first component of the codeBLEU score, we can find it here as well
    regular_BLEU = -1
    re_match = re.search(r"ngram match:\s+(\d\.\d+)", result.stdout)
    if re_match:
      regular_BLEU = float(re_match.group(1))


    codeBLEU = -1
    re_match = re.search(r"CodeBLEU score:\s+(\d\.\d+)", result.stdout)
    if re_match:
      codeBLEU = float(re_match.group(1))

  except subprocess.CalledProcessError as e:
    print(e.stderr)
    codeBLEU = -1

  return codeBLEU, regular_BLEU

# Do CodeBLEU Evaluation

In [56]:
os.makedirs("data", exist_ok=True)

get_codeBLEU_score(test.loc[0]["formatted_method"],
                   test.loc[0]["if_statement"],
                   test.loc[0]["predicted_statement"],
                   "/content/data/target.txt",
                   "/content/data/prediction.txt")

(0.8638060396705811, 0.8516228624291206)

In [57]:
from tqdm import tqdm

tqdm.pandas()

test[["CodeBLEU", "BLEU-4"]] = test.progress_apply(lambda row: pd.Series(get_codeBLEU_score(row["formatted_method"],
                                                             row["if_statement"],
                                                             row["predicted_statement"],
                                                             "/content/data/target.txt",
                                                             "/content/data/prediction.txt")), axis=1)
print("\nNumber of times failed to comupte CodeBLEU: ", (test["CodeBLEU"] == -1).sum())
test

100%|██████████| 4987/4987 [21:11<00:00,  3.92it/s]


Number of times failed to comupte CodeBLEU:  0


,Unnamed: 0,formatted_method,if_statement,predicted_statement,CodeBLEU,BLEU-4
0,0,"def read(self, count=True, timeout=None, ignor...",if ignore_timeouts and is_timeout(e):,if ignore_timeouts and is_noerr(e):,0.863806,0.851623
1,1,"def _cache_mem(curr_out, prev_mem, mem_len, re...",if prev_mem is None:,if prev_mem is None:,0.615985,0.887966
2,2,def filtered(gen): for example in gen: example...,if example_len > max_length:,if example_len > max_length:,0.879623,0.810593
3,3,"def search(self, query): if not query: logger....","if item.get('type', '') == 'audio':",if 'guide_id' in item:,0.888856,0.866107
4,4,"def _check_script(self, script, directive): fo...",if var.must_contain('/'):,if not var.can_match(directive):,0.715068,0.859290
...,...,...,...,...,...,...
4982,4982,"def _super_function(args): passed_class, passe...","if isinstance(pyclass, pyobjects.AbstractClass):","if isinstance(pyclass, pyobjects.PyObject):",0.874765,0.833079
4983,4983,"def get_data(row): data = [] for field_name, f...",if result:,if result:,0.773603,0.893525
4984,4984,"def say(jarvis, s): """"""Reads what is typed.""""""...",if not voice_state:,if voice_state:,0.642402,0.870610
4985,4985,"def __import__(name, globals=None, locals=None...",if '*' in fromlist:,if '*' in fromlist:,0.942053,0.885633


In [59]:
test["Exact Match"] = test["if_statement"] == test["predicted_statement"]
test

,Unnamed: 0,formatted_method,if_statement,predicted_statement,CodeBLEU,BLEU-4,Exact Match
0,0,"def read(self, count=True, timeout=None, ignor...",if ignore_timeouts and is_timeout(e):,if ignore_timeouts and is_noerr(e):,0.863806,0.851623,False
1,1,"def _cache_mem(curr_out, prev_mem, mem_len, re...",if prev_mem is None:,if prev_mem is None:,0.615985,0.887966,True
2,2,def filtered(gen): for example in gen: example...,if example_len > max_length:,if example_len > max_length:,0.879623,0.810593,True
3,3,"def search(self, query): if not query: logger....","if item.get('type', '') == 'audio':",if 'guide_id' in item:,0.888856,0.866107,False
4,4,"def _check_script(self, script, directive): fo...",if var.must_contain('/'):,if not var.can_match(directive):,0.715068,0.859290,False
...,...,...,...,...,...,...,...
4982,4982,"def _super_function(args): passed_class, passe...","if isinstance(pyclass, pyobjects.AbstractClass):","if isinstance(pyclass, pyobjects.PyObject):",0.874765,0.833079,False
4983,4983,"def get_data(row): data = [] for field_name, f...",if result:,if result:,0.773603,0.893525,True
4984,4984,"def say(jarvis, s): """"""Reads what is typed.""""""...",if not voice_state:,if voice_state:,0.642402,0.870610,False
4985,4985,"def __import__(name, globals=None, locals=None...",if '*' in fromlist:,if '*' in fromlist:,0.942053,0.885633,True


In [65]:
# Replace the path with wherever you want the testset_results CSV to go in your google drive
test.to_csv("/content/drive/MyDrive/AIFSD Lab 3/Homework 2/evaluation_data/testset_results.csv", index=False)

In [64]:
print("Average CodeBLEU Score:\t", test["CodeBLEU"].mean())
print("Average BLEU-4 Score:\t", test["BLEU-4"].mean())
print(f"Fraction Exact Match:\t {test['Exact Match'].sum()/len(test)}")

Average CodeBLEU Score:	 0.8066421660406118
Average BLEU-4 Score:	 0.8469100403997822
Fraction Exact Match:	 0.29075596551032684
